In [6]:
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

import numpy as np

import pandas as pd


In [7]:
# carregar modelo word2vec
word2vec = KeyedVectors.load_word2vec_format('../cbow_s100.txt')
vocab_size, embedding_size = word2vec.vectors.shape

In [8]:
# carregar dados de treinamento do CSV
df = pd.read_csv('../train.csv', header=None, names=['texto', 'classe'])
X = df['texto'].values
y = df['classe'].values

In [9]:
# tokenizar o texto
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# padronizar o tamanho das sequencias
max_length = max([len(text) for text in X])
X = pad_sequences(X, maxlen=max_length, padding='post')

In [36]:
# construir modelo
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, weights=[word2vec.vectors], input_length=max_length, trainable=False))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Dense(7, activation='softmax'))

In [37]:
# compilar modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
# separar dados em treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, 7)
y_val = to_categorical(y_val, 7)

In [39]:
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=7,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

In [40]:
# treinar modelo
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping_monitor])

Epoch 1/30
26/26 [==============================] - 3s 36ms/step - loss: 1.5059 - accuracy: 0.4495 - val_loss: 0.9178 - val_accuracy: 0.6355
Epoch 2/30
26/26 [==============================] - 0s 13ms/step - loss: 0.5285 - accuracy: 0.8325 - val_loss: 0.2541 - val_accuracy: 0.9360
Epoch 3/30
26/26 [==============================] - 0s 12ms/step - loss: 0.1706 - accuracy: 0.9606 - val_loss: 0.2072 - val_accuracy: 0.9507
Epoch 4/30
26/26 [==============================] - 0s 12ms/step - loss: 0.1535 - accuracy: 0.9507 - val_loss: 0.2135 - val_accuracy: 0.9409
Epoch 5/30
26/26 [==============================] - 0s 13ms/step - loss: 0.0879 - accuracy: 0.9766 - val_loss: 0.2194 - val_accuracy: 0.9360
Epoch 6/30
26/26 [==============================] - 0s 12ms/step - loss: 0.0762 - accuracy: 0.9803 - val_loss: 0.2646 - val_accuracy: 0.9360
Epoch 7/30
26/26 [==============================] - 0s 12ms/step - loss: 0.0501 - accuracy: 0.9877 - val_loss: 0.2054 - val_accuracy: 0.9557
Epoch 8/30
26